In [1]:
# 0-1 Knapsack Problem using Dynamic Programming (with selected items)

def knapsack(weights, values, capacity):
    n = len(values)
    
    # Step 1: Create DP table
    dp = [[0 for _ in range(capacity + 1)] for _ in range(n + 1)]

    # Step 2: Fill DP table
    for i in range(1, n + 1):
        for w in range(1, capacity + 1):
            if weights[i-1] <= w:
                dp[i][w] = max(values[i-1] + dp[i-1][w - weights[i-1]], dp[i-1][w])
            else:
                dp[i][w] = dp[i-1][w]

    # Step 3: Backtrack to find selected items
    selected_items = []
    w = capacity
    for i in range(n, 0, -1):
        if dp[i][w] != dp[i-1][w]:
            selected_items.append(i)  # item i was included
            w -= weights[i-1]

    selected_items.reverse()  # To show items in original order

    # Step 4: Return results
    return dp[n][capacity], selected_items


# Example usage
weights = [2, 3, 4, 5]
values = [3, 4, 5, 6]
capacity = 5

max_value, items = knapsack(weights, values, capacity)
print("Maximum value in Knapsack =", max_value)
print("Items included (1-indexed) =", items)

Maximum value in Knapsack = 7
Items included (1-indexed) = [1, 2]


In [5]:
# 0-1 Knapsack using Branch and Bound approach

# Class to represent an item with value, weight, and value-to-weight ratio
class Item:
    def __init__(self, value, weight):
        self.value = value
        self.weight = weight
        self.ratio = value / weight

# Node structure to represent a state in the solution space tree
class Node:
    def __init__(self, level, profit, weight, bound):
        self.level = level      # Index of current item
        self.profit = profit    # Current profit (value)
        self.weight = weight    # Current weight
        self.bound = bound      # Upper bound on possible profit from this node

# Function to calculate the upper bound (maximum possible profit) for a node
def bound(node, n, capacity, items):
    if node.weight >= capacity:
        return 0  # Can't include more items

    profit_bound = node.profit
    j = node.level + 1
    totweight = node.weight

    # Include items while capacity allows
    while j < n and totweight + items[j].weight <= capacity:
        totweight += items[j].weight
        profit_bound += items[j].value
        j += 1

    # Include a fraction of the next item (for bounding only)
    if j < n:
        profit_bound += (capacity - totweight) * items[j].ratio

    return profit_bound


# Main Branch and Bound function
def knapsack_branch_and_bound(values, weights, capacity):
    n = len(values)
    items = [Item(values[i], weights[i]) for i in range(n)]

    # Step 1: Sort items by value-to-weight ratio in descending order
    items.sort(key=lambda x: x.ratio, reverse=True)

    # Step 2: Initialize queue for BFS and variables
    queue = []
    u = Node(-1, 0, 0, 0)
    v = Node(-1, 0, 0, 0)
    max_profit = 0

    # Step 3: Compute bound for root node and enqueue
    u.bound = bound(u, n, capacity, items)
    queue.append(u)

    # Step 4: Process nodes in the queue
    while queue:
        u = queue.pop(0)  # Dequeue a node

        if u.level == n - 1:
            continue

        # Move to next level
        v.level = u.level + 1

        # CASE 1: Include next item
        v.weight = u.weight + items[v.level].weight
        v.profit = u.profit + items[v.level].value

        # Update maximum profit if valid
        if v.weight <= capacity and v.profit > max_profit:
            max_profit = v.profit

        v.bound = bound(v, n, capacity, items)
        if v.bound > max_profit:
            queue.append(Node(v.level, v.profit, v.weight, v.bound))

        # CASE 2: Exclude next item
        v.weight = u.weight
        v.profit = u.profit
        v.bound = bound(v, n, capacity, items)
        if v.bound > max_profit:
            queue.append(Node(v.level, v.profit, v.weight, v.bound))

    return max_profit


# ---------------- Example Run ----------------
values = [60, 100, 120]
weights = [10, 20, 30]
capacity = 50

print("0-1 KNAPSACK USING BRANCH AND BOUND")
print("-----------------------------------")
print("Items:")
for i in range(len(values)):
    print(f"  Item {i+1}: Value = {values[i]}, Weight = {weights[i]}, Ratio = {values[i]/weights[i]:.2f}")

print(f"\nKnapsack capacity = {capacity}")

result = knapsack_branch_and_bound(values, weights, capacity)
print(f"Maximum value in 0-1 Knapsack (Branch & Bound) = {result}")


0-1 KNAPSACK USING BRANCH AND BOUND
-----------------------------------
Items:
  Item 1: Value = 60, Weight = 10, Ratio = 6.00
  Item 2: Value = 100, Weight = 20, Ratio = 5.00
  Item 3: Value = 120, Weight = 30, Ratio = 4.00

Knapsack capacity = 50

📘 Explanation:
The algorithm explores a state-space tree of possible item selections.
Each node represents a partial solution with an upper bound on achievable profit.
Branches with bounds lower than the current best profit are pruned (ignored).
This reduces the number of combinations explored, improving efficiency.

Maximum value in 0-1 Knapsack (Branch & Bound) = 220
